<p style="color:darkred; font-family:candara; font-size:400%; text-align:center;"> 
    COVID-19
</p>

***

This notebook will study my own questions about the coronavirus phenomenon that has alarmed the world and that has waken us up about the implication of pandemias in a globalised world. 
[In this link](https://systems.jhu.edu/research/public-health/ncov/) you can find a nice review of how these events have developed since its first appearance.

I think (and hope) that the world won't devolve into xenofobic, rigidly separate countries with hard borders, hence after this analysis, the most important message to take home is that nations should act more tightly and in a synchronised manner when the next pandemia rises.

Whilst looking at the great map done by the [John Hopkins University](https://systems.jhu.edu/research/public-health/ncov/), I was left with so many more questions than answers about many details of this illness and how it is affecting each country. 
Hopefully some indagation could help understand better the spread and the impact of this disease.
<br><br>
<a href="https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6">
    <img src="images/COVID-19_global-cases.png" alt="COVID-19" width="650" >
</a>

*Source: Dong E, Du H, Gardner L. [An interactive web-based dashboard to track COVID-19 in real time](https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6). Lancet Infect Dis; published online Feb 19, and retrieved Apr 1, 2020. https://doi.org/10.1016/S1473-3099(20)30120-1.*

I will use the plots generated in this notebook to be shown on my [personal webpage](https://cuspime.github.io/).
Hopefully some indagation could help understand better the spread and the impact of this disease.

The data used here was taken from the [**John Hopkins University data on GitHub**](https://github.com/CSSEGISandData/COVID-19/tree/master/archived_data), which in turn is compiled from different sources. This database is updated once a day.

## Imports

In [1]:
import pandas as pd
from pandas import Grouper  # This is actually quite good to group dates
import numpy as np
import seaborn as sns; sns.set()
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import gridspec
import re
#------------------- Interactive imports:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from matplotlib.dates import DateFormatter
#------------------------ For modelling
from scipy.optimize import curve_fit
from datetime import datetime, timedelta

#==========================  Bokeh imports =======================
from bokeh.io import curdoc, push_notebook, output_notebook, reset_output
from bokeh.models import (DateSlider, ColumnDataSource, Button, Panel, Tabs,
                          CategoricalColorMapper, HoverTool, Select,
                          LinearColorMapper, BasicTicker, PrintfTickFormatter,
                          ColorBar, FactorRange)
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import gridplot, row, column, widgetbox
from bokeh.palettes import Category20, Blues8
from bokeh.models.glyphs import Text
from bokeh.transform import linear_cmap
import matplotlib.cm as cm

import sys

# output_notebook()
output_file('covid.html')

In [2]:
urlconfirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
urldeaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
urlrecovered = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

df = pd.read_csv(urlconfirmed, error_bad_lines=False)
deaths = pd.read_csv(urldeaths, error_bad_lines=False)
recov = pd.read_csv(urlrecovered, error_bad_lines=False)

In [3]:
#=========== Cosmetic changes
df = df.rename(columns={'Province/State':'provincestate', 'Country/Region':'countryregion', 'Lat':'lat', 'Long':'lon'}) #
recov = recov.rename(columns={'Province/State':'provincestate', 'Country/Region':'countryregion', 'Lat':'lat', 'Long':'lon'}) #
deaths = deaths.rename(columns={'Province/State':'provincestate', 'Country/Region':'countryregion', 'Lat':'lat', 'Long':'lon'}) #

latlon = df[['countryregion', 'lat', 'lon']] # We have to save lat and long 
# I am not interested in regions of the US. which seems to be the only one with meaningful province/state
df = df.groupby('countryregion', as_index=False).sum()
recov = recov.groupby('countryregion', as_index=False).sum()
deaths = deaths.groupby('countryregion', as_index=False).sum()

#=========== Important changes
#--------- Melting the df and creating columns and rows
df = pd.melt(df, id_vars=df.iloc[:,:3].columns.tolist(), value_vars=df.iloc[:,3:].columns.tolist(), var_name='date' )
df.date = pd.to_datetime(df.date, format='%m/%d/%y') # %y deals with only a 2 digit year
df = df.sort_values(by=['countryregion', 'date']) # Much better to have them ordered by country
deaths = pd.melt(deaths, id_vars=deaths.iloc[:,:3].columns.tolist(), value_vars=deaths.iloc[:,3:].columns.tolist(), var_name='date' )
deaths.date = pd.to_datetime(deaths.date, format='%m/%d/%y') # %y deals with only a 2 digit year
deaths = deaths.sort_values(by=['countryregion', 'date']) # Much better to have them ordered by country
recov = pd.melt(recov, id_vars=recov.iloc[:,:3].columns.tolist(), value_vars=recov.iloc[:,3:].columns.tolist(), var_name='date' )
recov.date = pd.to_datetime(recov.date, format='%m/%d/%y') # %y deals with only a 2 digit year
recov = recov.sort_values(by=['countryregion', 'date']) # Much better to have them ordered by country

#--------- Create 'World' entry with the sum of all countries (except China! as we'll discuss😊)
sum_of_countries = df[df.countryregion!='China'].groupby('date').value.sum()
frame = { 'countryregion': 'World', 'date':sum_of_countries.index, 'value': sum_of_countries.values} 
df = pd.concat([df, pd.DataFrame(frame) ], ignore_index=True, sort=False)
sum_of_countries_deaths = deaths[deaths.countryregion!='China'].groupby('date').value.sum()
frame_deaths = { 'countryregion': 'World', 'date':sum_of_countries_deaths.index, 'value': sum_of_countries_deaths.values} 
deaths = pd.concat([deaths, pd.DataFrame(frame_deaths) ], ignore_index=True, sort=False)
sum_of_countries_recov = recov[recov.countryregion!='China'].groupby('date').value.sum()
frame_recov = { 'countryregion': 'World', 'date':sum_of_countries_recov.index, 'value': sum_of_countries_recov.values} 
recov = pd.concat([recov, pd.DataFrame(frame_recov) ], ignore_index=True, sort=False)

#--------- Obtaining the delta difference from day to day (this can be done easily thanks to the order we have indicated) 👌
df['delta'] = df.iloc[:,3:].diff().value
deaths['delta'] = deaths.iloc[:,3:].diff().value
recov['delta'] = recov.iloc[:,3:].diff().value

#---------- Death percentage
df['deathperc'] = 100*deaths.value /(deaths[deaths.countryregion=='World'].value + recov[recov.countryregion=='World'].value)

#========= Countries considered in plots !!!!!!
countries = ['Mexico','Italy', 'Canada', 'Netherlands', 'Germany', 'United Kingdom', 'China', 'US', 'Korea, South']

In [4]:
#---------- Predictions
# Define the function to be used in the approximation
def test_func(x, a, b, c,d,e):
    return a* (1.0/(1 + np.exp(-b * x+c) ) )*np.exp(d*x + e) # Sigmoid logistic and exponenial

worldval = df[df.countryregion=='World'].value.values
# params, params_covariance = curve_fit(test_func, x_data, y_data, p0=[mean, sigma])
params, params_covariance = curve_fit(test_func, np.arange(0, len(worldval), 1), worldval,
                                      p0=[0.001,.0001,.0001, .00001, .00001])
ndaysahead = 5

date_range = df[df.countryregion=='World'].date
date_arr = np.array([ date_range.iloc[0]  + timedelta(i) for i in range(len(date_range)+ndaysahead+1) ]) # Used for the plot
x_pred = np.arange(len(worldval) + ndaysahead+1 ) # Used for the fit of the function 
list_predic = test_func(x_pred, params[0], params[1], params[2], params[3], params[4])
src_predict = ColumnDataSource(data = {'date':date_arr, 'value': list_predic } )



# Bokeh plots

In [5]:
# Necessary variables
lcolores=Category20[len(countries)]
color = dict(zip(countries[:len(countries)], lcolores) )
selected_scale = 'log'# Default Scale 

src_world = ColumnDataSource(data=df[df['countryregion'] == 'World'])
src_world_d = ColumnDataSource(data=deaths[deaths['countryregion'] == 'World'])
src_world_r = ColumnDataSource(data=recov[recov['countryregion'] == 'World'])

#---------- Setting the plots
# World totals
largo = 900
alto = 500
tools_s = "pan,wheel_zoom,box_zoom,reset,save"
p1 = figure(plot_width=largo, plot_height=alto, title='Worldwide total', x_axis_type='datetime', x_axis_label='Date', 
            y_axis_label='Number of people (log scale)', y_axis_type="log", y_range=(10,1.3*df.iloc[-1].value),
           tools=tools_s )
# World new cases
p2 = figure(plot_width=largo, plot_height=int(.75*alto), title='Worldwide new cases', x_axis_type='datetime', x_axis_label='Date', 
            y_axis_label='Number of people',
           tools=tools_s)
# Totals by country
p3 = figure(plot_width=largo, plot_height=alto, title='Totals by country', x_axis_type='datetime', x_axis_label='Date', 
            x_range=(df.date.tolist()[25], df.date.tolist()[-1]), y_axis_label='Number of people ({} scale)'.format(selected_scale), 
            y_axis_type=selected_scale, y_range=(100, 10e6), active_scroll='wheel_zoom',
           tools=tools_s)
# New cases by country
p4 = figure(plot_width=largo, plot_height=alto, title='New cases by country', x_axis_type='datetime', x_axis_label='Date', 
            y_axis_label='Number of people', x_range=(df.date.tolist()[25], df.date.tolist()[-1]), 
            y_range=[10,1.1*df[df.countryregion!='World'].delta.max() ], y_axis_type="log", 
            active_scroll='wheel_zoom',tools=tools_s )
# New cases vs Totals
p5 = figure(plot_width=largo, plot_height=alto, title='New cases by country', x_axis_label='Total number of cases (log scale)',
            y_axis_label='Number of new cases (log scale)', x_range=[10e2,2*10e5], y_range=[10,10e4], x_axis_type="log", 
            y_axis_type="log", 
            active_scroll='wheel_zoom', tools=tools_s )
# Totals by weekday
weekdays = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
df['weekday'] = df.date.dt.dayofweek
week_df = df[df.countryregion=='World'].groupby('weekday', as_index=False).sum()
src_weekday = ColumnDataSource(week_df)
p6 = figure(plot_width=largo, plot_height=alto, title='Totals by weekday (Last update on: {})'.format(weekdays[df.weekday.iloc[-1]]) , x_axis_label='Day of the week',
            y_axis_label='Total number of cases', tools=tools_s , y_range=[ .97 * src_weekday.data['delta'].min() ,1.03* src_weekday.data['delta'].max()])

# World death percentage
p7 = figure(plot_width=largo, plot_height=alto, title='Worlwide death percentage (100*deaths/ (deaths+recovered) )', x_axis_type='datetime', 
            x_axis_label='Date', x_range=(df.iloc[12].date,df.iloc[-1].date), y_axis_label='Percentage (%)', 
             y_range=(0, 45), tools=tools_s)

#==================== Plotting 
# p1
p1t_pred = p1.line( x='date', y = 'value', source = src_predict, color='blue', line_width=1.5, line_dash='dotdash', legend_label='Predicted', alpha=.7)
p1t = p1.line( x='date', y='value',line_width=2 , source = src_world, legend_label='Total')
p1r = p1.line( x='date', y='value',line_width=2 , source = src_world_r, color='green', legend_label='Recovered')
p1d = p1.line( x='date', y='value',line_width=2 , source = src_world_d, color='red', legend_label='Deaths')


# p2
p2.line( x='date', y='delta', source = src_world, line_width=.5, line_dash='dashed', color='gray', alpha=.7 )
p2.line( x='date', y='delta', source = src_world_d, line_width=.5, line_dash='dashed', color='red', alpha=.7 )
p2.line( x='date', y='delta', source = src_world_r, line_width=.5, line_dash='dashed', color='green', alpha=.7 )
p2t = p2.circle( x='date', y='delta', source = src_world, legend_label='Total')
p2r = p2.circle( x='date', y='delta', source = src_world_r, color='green', legend_label='Recovered')
p2d = p2.circle( x='date', y='delta', source = src_world_d, color='red', legend_label='Deaths')
p2.left[0].formatter.use_scientific = False 
#
for k in countries:
    src = ColumnDataSource(data=df[df.countryregion == k] )
    src_last = ColumnDataSource(data=df[df.countryregion == k].iloc[-1:] )
    
    p3.line( x='date', y='value', line_width=2.5, source = src, legend_label=k, line_color=color[k], muted_color=color[k], 
            muted_alpha=0.1, alpha=.9)
    p4.line( x='date', y='delta', line_width=2, source = src, legend_label=k, line_color=color[k], muted_color=color[k], 
            muted_alpha=0.1, alpha=.9 )
    p5.circle( x='value', y='delta', source = src, legend_label=k, size=3.5, color=color[k], muted_color=color[k], 
              muted_alpha=0.01, alpha=.3, fill_alpha=.1)
    p5.circle(  x='value', y='delta', source = src_last, size=10, color=color[k], alpha=.8 )

# p6
#Color control
mapper = linear_cmap(field_name='delta', palette=Blues8[::-1] ,low=week_df.delta.min() ,high=week_df.delta.max() )
p6.vbar( x='weekday', top='delta', source = src_weekday, fill_color=mapper,  alpha=.8, width=.9, line_width=3.5, fill_alpha=.55)
p6.xaxis.major_label_overrides = {i:weekdays[i] for i in range(len(weekdays))} 

# p7
p7.line( x='date', y='deathperc',line_width=2.5 , color='darkred', source = src_world)
p7.varea_stack(stackers=['deathperc'], x='date', source=src_world, color=('darkred'), alpha=.5 )


#========== Widgets & interactivity
p1.add_tools(HoverTool(renderers=[p1t_pred], tooltips=[('Date', '@date{%d-%b}'),('Pred' , '@value')], formatters={'@date': 'datetime'}, mode='vline' ))
p1.add_tools(HoverTool(renderers=[p1t], tooltips=[('Date', '@date{%d-%b}'),('Total' , '@value')], formatters={'@date': 'datetime'}, mode='vline' ))
p1.add_tools(HoverTool(renderers=[p1r], tooltips=[('Recov' , '@value')], formatters={'@date': 'datetime'}, mode='vline' ))
p1.add_tools(HoverTool(renderers=[p1d], tooltips=[('Dead' , '@value')], formatters={'@date': 'datetime'}, mode='vline' ))
p1.legend.background_fill_alpha = 0.7
p1.legend.click_policy="hide"
p1.legend.location = 'top_left'

p2.add_tools(HoverTool(renderers=[p2t], tooltips=[('Date', '@date{%d-%b}'),('Total' , '@value')], formatters={'@date': 'datetime'}, mode='vline' ))
p2.add_tools(HoverTool(renderers=[p2r], tooltips=[('Recov' , '@value')], formatters={'@date': 'datetime'}, mode='vline' ))
p2.add_tools(HoverTool(renderers=[p2d], tooltips=[('Dead' , '@value')], formatters={'@date': 'datetime'}, mode='vline' ))
p2.legend.background_fill_alpha = 0.7
p2.legend.click_policy="hide"
p2.legend.location = 'top_left'

p3.legend.click_policy="mute"
p3.legend.location = 'top_left'
p4.legend.click_policy="mute"
p4.legend.location = 'top_left'
p5.legend.click_policy="mute"
p5.legend.location = 'top_left'

hover3 = HoverTool(tooltips=[('Date', '@date{%d-%b}'),('Totals' , '$y'),('Country', '@countryregion')], formatters={'@date': 'datetime'} )
p3.add_tools(hover3)
p4.add_tools(hover3)
hover5 = HoverTool(tooltips=[('Date', '@date{%d-%b}'),('New cases' , '@delta'),('Total' , '@value'),('Country', '@countryregion')], formatters={'@date': 'datetime'} )
p5.add_tools(hover5)

hover6 = HoverTool(tooltips=[('Total number on this day' , '@delta') ] ,  mode='vline')
p6.add_tools(hover6)

p7.add_tools(HoverTool(tooltips=[('Date', '@date{%d-%b}'),('Percentage' , '@deathperc')], formatters={'@date': 'datetime'}, mode='vline' ))

#-------------  Callbacks

# # --------------------------------------------------------------------------------
# # To run and test things we need a button that will stop the server and kill the process
# def button_callback():
#     sys.exit()  # Stop the server
# # Button to stop the server
# exit_button = Button(label="Stop", button_type="warning", background='red', width=50)
# exit_button.on_click(button_callback)
# # --------------------------------------------------------------------------------
# lastrow = [exit_button]
# Text(x="x", y="y", text="text", angle=0.3, text_color="#96deb3")
lastrow = []

# Scale change
def change_scale(attr, old, new):
    selected_scale = seleccion.value
    p3 = figure(plot_width=largo, plot_height=alto, title='Totals by country', x_axis_type='datetime', x_axis_label='Date', x_range=(df.date.tolist()[25], df.date.tolist()[-1]), 
                y_axis_label='Number of people ({} scale)'.format(selected_scale), y_axis_type=selected_scale, y_range=(100, 10e6))
    p4 = figure(plot_width=largo, plot_height=alto, title='New cases by country', x_axis_type='datetime', x_axis_label='Date', y_axis_label='Number of people', x_range=(df.date.tolist()[25], df.date.tolist()[-1]), y_range=[0,1.1*df[df.countryregion!='World'].delta.max() ] )
    print(seleccion.value, new, 'Hello!!!')
    #-------------- Seems like we have to replot everything!!

def update_plot(attr,old, new):
    # Assign the value of the slider: yr
    maxdate = slider.value
    # Set new_data
    new_data = dict()
    new_data = {
        'index' : df[df['date']< maxdate].index.values,
        'countryregion' : df[df['date']< maxdate].countryregion.values,
        'date'       : df[df['date']< maxdate].date.values,
        'value'       : df[df['date']< maxdate].value.values,
        'delta'     : df[df['date']< maxdate].delta.values,
    }
    # Assign new_data to: source.data
    src.data = new_data
    # Add title to figure: plot.title.text
    plot.title.text = 'Last day plotted: %d' %maxdate

# Make a slider object: slider
slider = DateSlider(start=df.iloc[0].date, end=df.iloc[-1].date, step=1, value=df.iloc[-1].date, title='Day')

# Attach the callback to the 'value' property of slider
slider.on_change('value', update_plot)



#================= Heatmap
# Bokeh doesn't have its own gradient color maps supported but you can easily use on from matplotlib.
colormap =cm.get_cmap("coolwarm")
bokehpalette = [mpl.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))]
#this mapper_heatmap is what transposes a numerical value to a color. 
mapper_heatmap_min = df[df.countryregion=='World'].delta.iloc[1:].values.min()
mapper_heatmap_max = df[df.countryregion=='World'].delta.iloc[1:].values.max()
mapper_heatmap = LinearColorMapper(palette=bokehpalette, low=mapper_heatmap_min, high=mapper_heatmap_max )


groups = pd.Grouper(freq='M')
df[df.countryregion=='World'].groupby([pd.Grouper(key='date', freq='M')]).max() # By asking this to give the maximum, we basically ask the value of the last day of the month
df['day'] = df.date.dt.day
df['month'] = df.date.dt.month

src_w = ColumnDataSource( df[df.countryregion=='World'])

p_heatmap = figure(plot_width=largo, plot_height=int(.25*alto), x_axis_label='Day', y_axis_label='Month',# x_range=years, y_range=months, 
            toolbar_sticky=False, )
p_heatmap.rect(x='day', y='month', width=.99, height=.99, source=src_w,
        fill_color={'field': 'delta', 'transform': mapper_heatmap}, fill_alpha=.85,line_color=None)

p_heatmap.add_tools(HoverTool(tooltips=[('Date', '@date{%d-%b}'),('New cases' , '@delta')], formatters={'@date': 'datetime'} ))
p_heatmap.yaxis.visible = False
p_heatmap.grid.visible = False
# show(p_heatmap)

#=============== Tabs and Layouts
# seleccion = Select(title='Scale', options=['log', 'linear'], value='log', width=50)
# seleccion.on_change('value', change_scale)
layout1 = gridplot([ [p1], lastrow])
# layout2 = gridplot([ [p2], lastrow])
layout2 = gridplot([ [p2], [p_heatmap]])
layout3 = gridplot([ [p3], lastrow])
layout4 = gridplot([ [p4], lastrow])
# layout5 = gridplot([ [p5], [slider], lastrow ])
layout5 = gridplot([ [p5], lastrow])

tab1 = Panel(child=layout1, title="World totals")
tab2 = Panel(child=layout2, title="World new cases")
tab3 = Panel(child=layout3, title="Totals by country")
tab4 = Panel(child=layout4, title="New cases by country")
tab5 = Panel(child=layout5, title="New cases vs Totals")
tab6 = Panel(child=p6, title="Totals by weekday")
tab7 = Panel(child=p7, title="Death percentage")

tabs = Tabs(tabs=[ tab1, tab2, tab7, tab3, tab4, tab5, tab6])

show(tabs)
# show(layout5)
# curdoc().add_root(tabs)
# curdoc().add_root(layout5)

In [6]:
# This is the command one has to use on the Anaconda prompt
# !bokeh serve --show .\COVID19Bokeh.ipynb --port 5001

In [7]:
from bokeh.embed import file_html
from bokeh.resources import CDN
html = file_html( tabs, CDN, 'Covidtabs' )


In [8]:
print(html)





<!DOCTYPE html>
<html lang="en">
  
  <head>
    
      <meta charset="utf-8">
      <title>Covidtabs</title>
      
      
        
          
        
        
          
        <script type="text/javascript" src="https://cdn.bokeh.org/bokeh/release/bokeh-2.0.2.min.js" integrity="sha384-ufR9RFnRs6lniiaFvtJziE0YeidtAgBRH6ux2oUItHw5WTvE1zuk9uzhUU/FJXDp" crossorigin="anonymous"></script>
        <script type="text/javascript">
            Bokeh.set_log_level("info");
        </script>
        
      
      
    
  </head>
  
  
  <body>
    
      
        
          
          
            
              <div class="bk-root" id="8f307f1a-1882-4080-a341-53ce250cfb13" data-root-id="2648"></div>
            
          
        
      
      
        <script type="application/json" id="6349">
          {"e2418147-4a75-48a6-8e21-81280f62531d":{"roots":{"references":[{"attributes":{"data_source":{"id":"1439"},"glyph":{"id":"1468"},"hover_glyph":null,"muted_glyph":{"id":"1470"},"nonselec

In [9]:
# from scipy.stats.kde import gaussian_kde
# #Train the kernel
# ker = gaussian_kde(df[df.countryregion=='World'].delta[1:].values)
# #Select the points on where to evaluate
# ind = np.linspace(0, 1.5*10e4, 301 )
# #Evaluate the models at these points
# # plt.plot( ker.evaluate(ind) );
# # World death percentage
# p8 = figure(plot_width=largo, plot_height=alto//3, title=' Worldwide KDE for new cases',
#             x_axis_label='Number of new cases', x_range=(0,300), y_axis_label='Density', y_range=(0,0.000013),tools=tools_s)
# # p8
# p8.line( x=np.linspace(0, 300, 301 ), y=ker.evaluate(ind), line_width=2.5 )
# p8.varea_stack(stackers=['deathperc'], x='date', source=src_world, color=('darkred'), alpha=.5 )
# p8.add_tools(HoverTool(tooltips=[('Density' , '$y')] ))
# # show(p8)